In [1]:
import polars

df = polars.read_csv("./data/Trade_DetailedTradeMatrix.csv", encoding='latin-1')

In [59]:
colnames = []
for i in df.columns:
    i = i.replace(" ", "_")
    colnames.append(i)

df.columns = colnames

Reporter_Country_Code
Reporter_Country_Code_(M49)
Reporter_Countries
Partner_Country_Code
Partner_Country_Code_(M49)
Partner_Countries
Item_Code
Item_Code_(CPC)
Item
Element_Code
Element
Year_Code
Year
Unit
Value
Flag


This is the part where we analyze the data.

In [120]:
import duckdb

def item_dependency(country, year, item, threshold):
    query = '''
    select Partner_Countries,
        sum(Value) as trade_quantity,
        round(trade_quantity /
        (
            select
                sum(Value)
            from df
            where Year = {year}
            and Reporter_Countries = '{country}'
            and Item = '{item}'
            and Element = 'Import Quantity'
        ), 2) as dep_ratio
    from df
    where Year = {year}
    and Reporter_Countries = '{country}'
    and Element = 'Import Quantity'
    and Item = '{item}'
    and Value > 0
    group by Partner_Countries
    having dep_ratio >= {threshold}
    order by dep_ratio desc; 
'''.format(item = item, country = country, year = year, threshold = threshold)
    
    return duckdb.sql(query).pl()

item_dependency('United Arab Emirates', 2021, 'Bananas', 0.05)

Partner_Countries,trade_quantity,dep_ratio
str,f64,f64
"""Ecuador""",95247.0,0.64
"""India""",27022.2,0.18
"""Philippines""",23342.0,0.16
